In [ ]:
%pip install umap-learn
%pip install kmapper
%pip install scikit-learn

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import manifold
import umap
from kmapper import KeplerMapper
from sklearn.preprocessing import MinMaxScaler
import kmapper as km
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
import joblib
# Cargar los datos desde el archivo CSV
df = pd.read_csv('C:/Users/danie/Desktop/proyecto final/Microdatos_Aspirantes_y_Admitidos_en_la_UNAL_20240311.csv')

df = df[(df['NIVEL'] == 'Pregrado') & (df['PTOTAL'] > 10)]

# Columnas a eliminar del DataFrame
columnas_a_eliminar = ['RANGO_ANO_TERMINACION', 'MODALIDAD', 'AREA_CINE',
                       'COD_CIU_RES', 'COD_DEP_NAC', 'CD_CINE', 'SEMESTRE',
                       'CA_CINE', 'AREAC_SNIES', 'PROGRAMA', 'SNIES_PROGRA',
                       'INS_SEDE_NOMBRE', 'SNIES_SEDE', 'PEAMA', 'PAES',
                       'TIPO_INS', 'MOD_INS', 'TIPO_DISC', 'CODS_NAC',
                       'NACIONALIDAD', 'LAT_CIU_RES', 'LON_CIU_RES',
                       'TIPO_NIVEL', 'DEP_NAC', 'CIU_NAC', 'COD_CIU_NAC',
                       'LAT_CIU_NAC', 'LON_CIU_NAC', 'CIU_RES', 'DEP_RES',
                       'COD_DEP_RES', 'YEAR', 'CODN_NAC', 'NIVEL']

# Eliminar las columnas especificadas
data = df.drop(columnas_a_eliminar, axis=1)

# Definir el mapeo de valores para 'ESTRATO'
mapeo_estrato = {'ND/NE': 0, 'Estrato 1': 1, 'Estrato 2': 2, 'Estrato 3': 3, 'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6}
# Aplicar el mapeo a la columna 'ESTRATO'
data['ESTRATO'] = data['ESTRATO'].replace(mapeo_estrato)

# Calcular la distribución de los estratos existentes en los datos
distribucion_estratos = data['ESTRATO'].value_counts(normalize=True)

# Remover el estrato '0' de la distribución
distribucion_estratos_sin_cero = distribucion_estratos.drop(0)

# Normalizar las probabilidades para asegurarse de que sumen 1
probabilidades_normalizadas = distribucion_estratos_sin_cero / distribucion_estratos_sin_cero.sum()

# Elegir aleatoriamente un estrato basado en la distribución existente
nuevo_estrato = np.random.choice(probabilidades_normalizadas.index, p=probabilidades_normalizadas.values)

# Reemplazar los valores de '0' en la columna 'ESTRATO' con el nuevo estrato seleccionado
data.loc[data['ESTRATO'] == 0, 'ESTRATO'] = nuevo_estrato

# # Definir el mapeo de valores para SEXO
mapeo_sexo = {'Mujeres': 0, 'Hombres': 1, 'Transgénero': 2}

# # Aplicar el mapeo a la columna 'SEXO'
data['SEXO'] = data['SEXO'].replace(mapeo_sexo)

# Definir el mapeo de valores para 'DISCAPACIDAD'
mapeo_discapacidad = {'Sí': 0, 'No': 1}
# Aplicar el mapeo a la columna 'DISCAPACIDAD'
data['DISCAPACIDAD'] = data['DISCAPACIDAD'].replace(mapeo_discapacidad)

mapeo_admitido = {'Sí': 0, 'No': 1}
# Aplicar el mapeo a la columna 'ADMITIDO'
data['ADMITIDO'] = data['ADMITIDO'].replace(mapeo_admitido)
data.info()

# Separar las características (X) y la variable objetivo (y)
X = data[['EDAD', 'SEXO', 'ESTRATO', 'DISCAPACIDAD', 'PTOTAL']]
y = data['ADMITIDO']

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar las características
scaler = preprocessing.MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# param_grid = {
#     'hidden_layer_sizes': [(10,10)],
#     'activation': ['tanh', 'relu'],
#     'solver': ['sgd', 'adam'],
#     'alpha': [0.0001],
#     'learning_rate': ['adaptive'],
#     'learning_rate_init': [0.01],
#     'max_iter': [200]
# }

# # Inicializar el clasificador MLP
# mlp = MLPClassifier()

# # Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
# grid_search = GridSearchCV(mlp, param_grid, cv=5, verbose=1, n_jobs=-1)
# grid_search.fit(X_train_scaled, y_train)

# # Mostrar los mejores hiperparámetros encontrados
# print("Mejores hiperparámetros encontrados:")
# print(grid_search.best_params_)

# # Calcular la precisión en el conjunto de prueba con los mejores hiperparámetros
# accuracy = grid_search.best_estimator_.score(X_test_scaled, y_test)
# print("Precisión en el conjunto de prueba con mejores hiperparámetros:", accuracy)

# joblib.dump(grid_search.best_estimator_, 'modelo_mlp.joblib')

<class 'pandas.core.frame.DataFrame'>
Index: 417490 entries, 0 to 493794
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   EDAD          417490 non-null  int64  
 1   SEXO          417490 non-null  int64  
 2   ESTRATO       417490 non-null  int64  
 3   DISCAPACIDAD  417490 non-null  int64  
 4   PTOTAL        417490 non-null  float64
 5   ADMITIDO      417490 non-null  int64  
dtypes: float64(1), int64(5)
memory usage: 22.3 MB


In [10]:
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


clf = svm.SVC(C=0.001, kernel='poly', degree=5, gamma='scale', 
        coef0=0.0, shrinking=True, probability=False, 
        tol=0.00001, cache_size=500, class_weight=None, 
        verbose=True, max_iter=500, decision_function_shape='ovr', 
        break_ties=True, random_state=None)

clf.fit(X_train_scaled, y_train) 

# Predecir las etiquetas de salida para los datos de prueba
y_pred = clf.predict(X_test_scaled)


# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo SVM: {:.2f}".format(accuracy))

joblib.dump(clf, 'svm_model.joblib')

[LibSVM]

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Precisión del modelo SVM: 0.85


['svm_model.joblib']

In [ ]:
#Muestrear los datos
data = data.sample(n=100000, random_state=42)

# Convertir DataFrame a un arreglo de numpy
data_array = data.values
y = data_array[:, 4].astype(float)
z = data_array[:, 1].astype(float)
w = data_array[:, 2].astype(float)
# Inicialización de KeplerMapper
mapper = km.KeplerMapper()

# Ajuste y transformación de datos
projected_data = mapper.fit_transform(data_array,
                                      projection= [manifold.Isomap(n_components=3, n_jobs=-1),
                                                   umap.UMAP(n_components=2, random_state=1)],
                                      scaler=[ MinMaxScaler(), MinMaxScaler()])
#[manifold.Isomap(n_components=3, n_jobs=-1),
#  [None, MinMaxScaler()]   umap.UMAP(n_components=2, random_state=1)]
# Creación de un cubrimiento (cover)
cover = km.Cover(n_cubes=15, perc_overlap=0.6)  # Aumentar el número de cubos para cubrir mejor los datos

# Mapeo de los datos con una distancia más grande
graph = mapper.map(projected_data, data_array, cover=cover, clusterer=km.cluster.DBSCAN(eps=10, min_samples=50))
color_values = np.c_[y, z, w]
columna_combinada = y * 100 + z * 10 + w
mapper.visualize(
    graph,
    path_html="universidad_nacional_output.html",
    title="Mapeo de datos de la Universidad Nacional",
    custom_tooltips=columna_combinada,
    color_values=color_values,
    color_function_name= ["ADMITIDO","SEXO", "ESTRATO"],
    node_color_function=["mean", "std", "median", "max" ]
)
# html = mapper.visualize(graph, color_values=color_values, path_html="universidad_nacional_output.html",
#                         title="Mapeo de datos de la Universidad Nacional")

print("Visualización generada con éxito.")